In [ ]:
import os
import numpy as np
import cupy as cp
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog
from concurrent.futures import ThreadPoolExecutor

# コンスタントの定義
VU_REF_DB = -18.0  # 0 VU を -18 dBFS に設定
REF_LEVEL = 10**(VU_REF_DB / 20)

# ファイル選択ダイアログを表示してファイルを選択する
root = Tk()
root.withdraw()  # メインウィンドウを表示しない
file_paths = filedialog.askopenfilenames(filetypes=[("WAV files", "*.wav")])
root.destroy()  # Tkinterウィンドウを破棄する

# 保存先フォルダを選択
root = Tk()
root.withdraw()
save_directory = filedialog.askdirectory()
root.destroy()

# VU メーターフィルタ（約 300ms のリリースタイム）
def vu_meter(signal, rate, release_time=0.3):
    alpha = cp.exp(-1.0 / (release_time * rate))
    vu_level = cp.maximum.accumulate(alpha * cp.maximum.accumulate(cp.abs(signal)))
    return cp.asnumpy(vu_level)

# VU メーターのレベルを計算
def calculate_vu_levels(normalized_data, rate):
    vu_levels = np.zeros_like(normalized_data)
    for ch in range(normalized_data.shape[1]):
        vu_levels[:, ch] = vu_meter(cp.array(normalized_data[:, ch]), rate)
    return vu_levels

# 誤差逆伝播法でゲインを調整
def adjust_gain(normalized_data, rate, ref_level=REF_LEVEL, lr=0.01, tol=1e-4, max_iter=1000):
    gain = cp.array([1.0])  # 初期ゲイン
    for _ in range(max_iter):
        vu_levels = calculate_vu_levels(normalized_data * gain, rate)
        max_vu = np.max(vu_levels)
        error = max_vu - ref_level
        if np.abs(error) < tol:
            break
        gain_grad = cp.sign(error)  # 勾配計算
        gain -= lr * gain_grad  # 勾配降下ステップ
    return gain.item()

# 各ファイルに対して処理を実行
def process_file(file_path):
    # オーディオファイルの読み込み
    rate, data = wav.read(file_path)
    if data.ndim == 1:
        data = np.expand_dims(data, axis=1)  # モノラルファイルの場合、チャンネルを追加

    # サンプルデータを -1～1 の範囲に正規化
    data = data.astype(np.float32) / np.iinfo(np.int16).max
    print(f"Normalized data range: {np.min(data)} to {np.max(data)}")

    gain_adjustment = adjust_gain(data, rate)
    print(f"Gain adjustment factor for {os.path.basename(file_path)}: {gain_adjustment}")

    # 統一したゲインを適用
    adjusted_data = data * gain_adjustment
    print(f"Adjusted data range: {np.min(adjusted_data)} to {np.max(adjusted_data)}")

    # 保存先ファイルパスを生成
    output_path = os.path.join(save_directory, os.path.basename(file_path))

    # 調整後のオーディオデータを保存
    adjusted_data_int = np.int16(adjusted_data * np.iinfo(np.int16).max)
    wav.write(output_path, rate, adjusted_data_int)
    print(f"Adjusted audio saved as '{output_path}'")

    # プロットで結果を表示
    plt.figure(figsize=(10, 4))
    plt.plot(data[:, 0], label='Original Left')
    if data.shape[1] > 1:
        plt.plot(data[:, 1], label='Original Right')
    plt.plot(adjusted_data[:, 0], label='Adjusted Left')
    if adjusted_data.shape[1] > 1:
        plt.plot(adjusted_data[:, 1], label='Adjusted Right')
    plt.legend()
    plt.title(f'Audio Waveform for {os.path.basename(file_path)}')
    plt.show()

    plt.figure(figsize=(10, 4))
    vu_levels = calculate_vu_levels(adjusted_data, rate)
    plt.plot(vu_levels[:, 0], label='VU Levels Left')
    if vu_levels.shape[1] > 1:
        plt.plot(vu_levels[:, 1], label='VU Levels Right')
    plt.axhline(y=REF_LEVEL, color='r', linestyle='--', label='0 VU (-18 dBFS)')
    plt.legend()
    plt.title(f'VU Meter Levels for {os.path.basename(file_path)}')
    plt.show()

# 並列処理を使用して各ファイルを処理
with ThreadPoolExecutor() as executor:
    executor.map(process_file, file_paths)
